# Kerchunk GRIDMET data
Using NetCDF files on s3

In [1]:
import numpy as np
import xarray as xr
import fsspec
from pathlib import Path
import ujson
import os

In [2]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr

In [3]:
#url = '/caldera/projects/usgs/water/wbeep/onhm_dev/climate/gridmet_raw'
fs = fsspec.filesystem('s3', requester_pays=True)

In [4]:
%%time
#flist = fs_local.glob('/caldera/projects/usgs/water/wbeep/onhm_dev/climate/gridmet_raw_rechunk/*_nc4.nc')
flist = sorted(fs.glob('s3://nhgf-development/gridmet/nc/*.nc'))
flist

CPU times: user 67 ms, sys: 60.1 ms, total: 127 ms
Wall time: 531 ms


['nhgf-development/gridmet/nc/gridmet_ppt_1979-2020_nc4.nc',
 'nhgf-development/gridmet/nc/gridmet_tmax_1979-2020_nc4.nc',
 'nhgf-development/gridmet/nc/gridmet_tmin_1979-2020_nc4.nc']

In [5]:
json_dir = 'jsons'

In [6]:
so = dict(mode='rb', requester_pays=True, default_fill_cache=False, default_cache_type='first')

In [7]:
fs_local = fsspec.filesystem('file')

In [12]:
def gen_json(u):
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        fstem = Path(u).stem 
        outf = f'{json_dir}/{fstem}.json'
        print(outf)
        with fs_local.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

#### Create the Kerchunk JSON for each individual file

In [13]:
%%time
for f in flist:
    gen_json(f)

jsons/gridmet_ppt_1979-2020_nc4.json
jsons/gridmet_tmax_1979-2020_nc4.json
jsons/gridmet_tmin_1979-2020_nc4.json
CPU times: user 8.05 s, sys: 1.52 s, total: 9.57 s
Wall time: 1min 34s


#### Check one to see if they look okay

In [14]:
rpath = 's3://esip-qhub/usgs/nhgf/gridmet/combined.json'
s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'requester_pays':True}
fs2 = fsspec.filesystem("reference", fo=rpath, ref_storage_args=s_opts,
                       remote_protocol='s3', remote_options=r_opts)

In [15]:
fs2.ls('')

[{'name': '.zgroup', 'type': 'file', 'size': 24},
 {'name': '.zattrs', 'type': 'file', 'size': 3439},
 {'name': 'day', 'type': 'directory', 'size': 0},
 {'name': 'lat', 'type': 'directory', 'size': 0},
 {'name': 'lon', 'type': 'directory', 'size': 0},
 {'name': 'precipitation_amount', 'type': 'directory', 'size': 0},
 {'name': 'daily_minimum_temperature', 'type': 'directory', 'size': 0},
 {'name': 'daily_maximum_temperature', 'type': 'directory', 'size': 0}]

In [30]:
fs2.ls('./daily_maxium_temperature')

FileNotFoundError: 

In [19]:
m = fs2.get_mapper("")
ds = xr.open_dataset(m, engine="zarr",chunks={'day': 1220, 'lat': 98, 'lon': 231},
                    backend_kwargs={'consolidated':False})

/home/conda/store/dbc7b9bf070bce5d6f873cfa3bf50d31adf596fb7e74657ab9f92206e92f0702-pangeo/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'precipitation_amount' has multiple fill values {32769, 32767}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [21]:
ds

<xarray.Dataset>
Dimensions:               (day: 15341, lat: 585, lon: 1386)
Coordinates:
  * day                   (day) datetime64[ns] 1979-01-01 ... 2020-12-31
  * lat                   (lat) float64 49.4 49.36 49.32 ... 25.15 25.11 25.07
  * lon                   (lon) float64 -124.8 -124.7 -124.7 ... -67.1 -67.06
Data variables:
    precipitation_amount  (day, lat, lon) float32 dask.array<chunksize=(1220, 98, 231), meta=np.ndarray>
Attributes: (12/21)
    Conventions:                CF-1.0
    History:                    Thu Nov 11 15:17:39 2021: ncks -O -4 -L 2 --c...
    NCO:                        netCDF Operators version 4.9.7 (Homepage = ht...
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 March 2020
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...